In [121]:

from src.data.dataset_reader import DatasetReader
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', None)
%load_ext autoreload
%autoreload 2
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
data_folder = Path("../data/raw").resolve()

amazon = DatasetReader(data_folder).get_data_csv('Amazon_Sale_Report.csv')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
from src.data.feature_engineering import FeatureEngineeringProcess
fep = FeatureEngineeringProcess()


In [73]:
# Create a new feature called sales that is the  the count of "Amount" grouped by "SKU" in the amazon dataset
#amazon['sales'] = amazon.groupby('SKU')['Amount'].transform('count')
#amazon.drop(columns=[ 'Order ID', 'ASIN', 'Unnamed: 22', 'index'], inplace=True
amazon['Date'] = pd.to_datetime(amazon['Date'])
amazon['ship-postal-code'] = amazon['ship-postal-code'].astype('object')
amazon_beta =  amazon[['Order ID','Date', 'SKU', 'Qty', 'Amount']]

In [75]:
amazon.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [76]:
amazon_beta.head()

,Order ID,Date,SKU,Qty,Amount
0,405-8078784-5731545,2022-04-30,SET389-KR-NP-S,0,647.62
1,171-9198151-1101146,2022-04-30,JNE3781-KR-XXXL,1,406.00
2,404-0687676-7273146,2022-04-30,JNE3371-KR-XL,1,329.00
3,403-9615377-8133951,2022-04-30,J0341-DR-L,0,753.33
4,407-1069790-7240320,2022-04-30,JNE3671-TU-XXXL,1,574.00


In [77]:
from src.data.eda import Eda
eda = Eda(    
)

eda.missing_values_table(amazon_beta)

The selected dataframe has 5 columns and 1 columns with missing values.


,Missing Values,% of Total Values
Amount,7795,6.0


In [78]:
amazon_beta.rename(columns={'Amount': 'price'}, inplace=True)

In [79]:
toy = amazon_beta.query('SKU == "SET389-KR-NP-S"').sort_values(by='Date', ascending=True)
toy['sales'] = toy.groupby(['SKU', 'Date', 'price'])['Order ID'].transform('count')

In [80]:
amazon_beta.head()

,Order ID,Date,SKU,Qty,price
0,405-8078784-5731545,2022-04-30,SET389-KR-NP-S,0,647.62
1,171-9198151-1101146,2022-04-30,JNE3781-KR-XXXL,1,406.00
2,404-0687676-7273146,2022-04-30,JNE3371-KR-XL,1,329.00
3,403-9615377-8133951,2022-04-30,J0341-DR-L,0,753.33
4,407-1069790-7240320,2022-04-30,JNE3671-TU-XXXL,1,574.00


In [81]:

# Create a column called "sales" that is the count of the "Order ID" grouped by "SKU" and "Date" and price in the amazon_beta dataset
amazon_beta['sales'] = amazon_beta.groupby(['SKU', 'Date', 'price'])['Order ID'].transform('count')


In [82]:
amazon_beta.head()

,Order ID,Date,SKU,Qty,price,sales
0,405-8078784-5731545,2022-04-30,SET389-KR-NP-S,0,647.62,1.0
1,171-9198151-1101146,2022-04-30,JNE3781-KR-XXXL,1,406.00,3.0
2,404-0687676-7273146,2022-04-30,JNE3371-KR-XL,1,329.00,2.0
3,403-9615377-8133951,2022-04-30,J0341-DR-L,0,753.33,2.0
4,407-1069790-7240320,2022-04-30,JNE3671-TU-XXXL,1,574.00,1.0


In [83]:
amazon_beta.sort_values(by=['SKU', 'Date'], ascending=True, inplace=True)

In [113]:

# drop Nan values
amazon_beta.dropna(inplace=True)

In [114]:
amazon_beta_fe1 = fep.price_sales_correlation_features_updated(amazon_beta, 10, [(-5.0, 1.0), (-3.0, 1.0), (-2.0, 1.0), (1.0, 1.0), (-1.0, 0.5), (-1.0, 0.33)])


In [130]:

amazon_beta_fe1.query('SKU == "SET389-KR-NP-S"').sort_values(by='Date', ascending=True).head()
# calculate average number of Date in each SKU
#amazon_beta_fe1.groupby('SKU')['Date'].nunique().mean()
# add a column called "days" that is the count of "Date" of each SKU
#amazon_beta_fe1['days'] = amazon_beta_fe1.groupby('SKU')['Date'].transform('nunique')
#amazon_beta_fe1.drop(columns=['days'], inplace=True)


,Order ID,Date,SKU,Qty,price,sales,f_corr_-5.0_1.0,f_corr_-3.0_1.0,f_corr_-2.0_1.0,f_corr_1.0_1.0,f_corr_-1.0_0.5,f_corr_-1.0_0.33
47784,406-4493775-5535509,2022-04-01,SET389-KR-NP-S,1,749.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN
41605,408-5859715-9071508,2022-04-05,SET389-KR-NP-S,1,0.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN
36996,402-3547090-0699503,2022-04-08,SET389-KR-NP-S,1,680.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN
18468,405-2959500-2916317,2022-04-19,SET389-KR-NP-S,1,680.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN
16932,406-8338408-6778764,2022-04-20,SET389-KR-NP-S,0,647.62,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
amazon_beta_fe2 = fep.normalize_features(amazon_beta, [5, 10, 15], 28)

In [127]:
amazon_beta_fe2

,Order ID,Date,SKU,Qty,price,sales,normalized_avg_price_5_days,normalized_avg_sales_5_days,normalized_log_avg_price_5_days,normalized_log_avg_sales_5_days,normalized_std_price_5_days,normalized_std_sales_5_days,normalized_avg_price_10_days,normalized_avg_sales_10_days,normalized_log_avg_price_10_days,normalized_log_avg_sales_10_days,normalized_std_price_10_days,normalized_std_sales_10_days,normalized_avg_price_15_days,normalized_avg_sales_15_days,normalized_log_avg_price_15_days,normalized_log_avg_sales_15_days,normalized_std_price_15_days,normalized_std_sales_15_days
98144,407-9280687-5435505,2022-06-23,SET462-KR-NP-S,1,999.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
#amazon_beta_fe2.head()
toy_sky = amazon_beta.query('SKU == "SET389-KR-NP-S"').sort_values(by='Date', ascending=True)

In [117]:
toy_sky = fep.normalize_features(toy_sky, [5, 10, 15], 28)

In [ ]:
amazon_beta_fe3, insuficient_data = fep.filter_stability_periods(amazon_beta_fe2, 7, 0.04)

In [ ]:
amazon_beta_fe3.head(10)


In [ ]:
# print without index amazon_beta_fe3.query('SKU == "SET389-KR-NP-S"') without index column


#amazon_beta_fe3[['Order ID', 'Date', 'SKU', 'price', 'sales','avg_price_last_7_days', 'price_variation']].query('SKU == "SET389-KR-NP-S"')


# NOTE: Still seeing repeated values in the "price" column. Either low the threshold or increase removal_percentage



In [ ]:

eda.missing_values_table(amazon_beta_fe3)

In [ ]:
# drop rows where price is null
amazon_beta_train = amazon_beta_fe3.dropna(subset=['price'])

In [ ]:
amazon_beta_train.fillna(0, inplace=True)

In [ ]:
amazon_beta_train1= fep.datetime_transform(amazon_beta_train, 'Date',['day_of_the_year', 'month', 'day_of_the_week'])

In [ ]:
# Drop Order Id, price_variation, insufficient_data
amazon_beta_train1.drop(columns=['Order ID', 'price_variation', 'insufficient_data'], inplace=True)

In [ ]:
#amazon_beta_train1.head()
amazon_beta_train1.drop(columns=['Date_season'], inplace=True)

In [ ]:
from src.data.data_splitter import DataSplitter

splitter = DataSplitter(amazon_beta_train1, train_months=1, test_weeks=2)
train, test = splitter.split_data()

In [ ]:
train.shape, test.shape
# print min and max dates of train and test
train['Date'].min(), train['Date'].max(), test['Date'].min(), test['Date'].max()
# Min Date: 2022-03-31 00:00:00, Max Date: 2022-06-29 00:00:00, Total Months: 3

In [ ]:
# Drop Date column
train.drop(columns=['Date'], inplace=True)
test.drop(columns=['Date'], inplace=True)
# one hot encode Date_season
#train = pd.get_dummies(train, columns=['Date_season'])
#test = pd.get_dummies(test, columns=['Date_season'])

In [ ]:
# Clean memory usage and delete dataframes that are not needed
#import gc
#del amazon, amazon_beta, amazon_beta_fe1, amazon_beta_fe2, amazon_beta_fe3, amazon_beta_train, amazon_beta_train1
#gc.collect()



In [ ]:
# dump as pickel files train and test dataframes in the data/processed folder
train.to_pickle('../data/processed/train.pkl')
test.to_pickle('../data/processed/test.pkl')

In [ ]:
# load train and test pickle from the data/processed folder
train = pd.read_pickle('../data/processed/train.pkl')

In [ ]:
# load test pickle from the data/processed folder
test = pd.read_pickle('../data/processed/test.pkl')

In [ ]:
train.columns

In [ ]:
import psutil

# Get memory usage in bytes
memory_usage = psutil.virtual_memory()
total_memory = memory_usage.total
available_memory = memory_usage.available
used_memory = memory_usage.used
percent_memory_used = memory_usage.percent

# Convert memory values to human-readable format
total_memory_readable = psutil._common.bytes2human(total_memory)
available_memory_readable = psutil._common.bytes2human(available_memory)
used_memory_readable = psutil._common.bytes2human(used_memory)

# Print memory usage information
print(f"Total Memory: {total_memory_readable}")
print(f"Available Memory: {available_memory_readable}")
print(f"Used Memory: {used_memory_readable}")
print(f"Percent Memory Used: {percent_memory_used}%")


In [ ]:
may = 467
june = 467
july = 497
august = 498
september = 515
october = 500
november = 515
december = 523

# get total sum of sales for each month

total_sales = may + june + july + august + september + october + november + december


In [ ]:
import modal

stub = modal.Stub("example-get-started")
@stub.function()
def f():
    return "hello"

with stub.run():
    assert "hello" == f.call()  # remote call works ✔️
    f()  # raises error in notebook ⚠️

In [ ]:
#check version of pandas
pd.__version__

In [ ]:
train.shape, test.shape

In [ ]:
test.columns

In [ ]:
train.columns

In [ ]:
test.shape

In [ ]:
import pandas as pd
predicted_sales = pd.read_csv('../data/processed/predictions.csv')

In [ ]:

predicted_sales.rename(columns={'predicted_sales_xgboost': 'predicted_sales'}, inplace=True)

In [ ]:
predicted_sales.head()

In [ ]:
input_df = predicted_sales.copy()

In [ ]:
input_df = input_df.head(1)

In [ ]:
input_df

In [ ]:
from src.pricing.pricing_algorithm import grid_search_lambda
import numpy as np

# Function to find optimal price for a given SKU
def find_optimal_price(sku_row, c, s, P0):
    # Extracting SKU, price and predicted sales
    sku = sku_row['SKU']
    price = sku_row['price']
    predicted_sale = sku_row['predicted_sales']
    
    print(f'\nProcessing SKU: {sku}')
    print(f'Original Price: {price}, Predicted Sales: {predicted_sale}')
    
    # Call grid_search_lambda function to find optimal price for the current SKU
    prices = np.array([price])
    predicted_sales = np.array([predicted_sale])
    
    print(f'Starting grid search for SKU: {sku}')
    best_prices, best_lambdas = grid_search_lambda(P0, prices, predicted_sales, c, s)
    
    # Extract the optimal price
    optimal_price = best_prices[0]
    
    print(f'Optimal Price for SKU {sku}: {optimal_price}\n')
    
    # Return the optimal price
    return optimal_price

# Parameters (you should calibrate these values based on your business case)
c = 10
s = 2
P0 = 100

# Assuming that input_df is the DataFrame that contains your data with columns: 'SKU', 'price', 'predicted_sales'

# Applying the function to each row in the DataFrame
optimal_prices = input_df.apply(find_optimal_price, args=(c, s, P0), axis=1)

# Creating a new DataFrame with SKU and optimal_prices
result_df = pd.DataFrame({'SKU': input_df['SKU'], 'optimal_prices': optimal_prices})

# Display the resulting DataFrame
print('\nFinal Result:')
print(result_df)


In [ ]:
lambda_min=3
lambda_max=5
lambda_step=1
range(lambda_min, lambda_max + 1, lambda_step)

In [ ]:
import pandas as pd
predictions = pd.read_csv('../data/processed/predictions.csv')
optimal_prices = pd.read_csv('../data/processed/optimal_prices.csv')

In [ ]:
optimal_prices.tail(10)

In [ ]:
optimal_prices.optimal_prices.value_counts()

In [ ]:
print(optimal_prices.query('SKU == "SET389-KR-NP-S"').to_string(index=False))

In [ ]:
print(predictions.query('SKU == "SET389-KR-NP-S"').to_string(index=False))

In [ ]:
amazon.query('SKU == "SET389-KR-NP-S"').Amount.value_counts()